# Uncertainty-Aware Road Obstacle Identification

## 1. Imports
Import necessary libraries like PyTorch, torchvision, etc.

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'torchvision'

## 2. Globals
Define global variables such as paths, batch size, learning rate, etc.

In [ ]:
# Global variables
DATASET_PATH = "./datasets/"
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCHS = 50

# Select the best available device (CPU, CUDA, or MPS (Metal Performance Shaders for macOS))
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using CUDA device: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Metal Performance Shaders) device")
else:
    device = torch.device("cpu")
    print("Using CPU")
DEVICE = device

## 3. Utils
Helper functions for visualization, preprocessing, and more.

In [ ]:

def visualize_sample(image, mask):
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Image")
    plt.imshow(image)
    plt.subplot(1, 2, 2)
    plt.title("Mask")
    plt.imshow(mask)
    plt.show()


## 4. Data
Load and preprocess datasets like Cityscapes, LostAndFound, and Fishyscapes.

In [ ]:
# Code for data loading and preprocessing

## 5. Network
Define the neural network architecture for semantic segmentation.

In [ ]:
# Define the model architecture here

# Lets replicate the Unknown Objectness Scores setup

# Load a pretrained DeepLabv3 model with a ResNet-50 backbone
model = models.segmentation.deeplabv3_resnet50(pretrained=True)
model.eval()  # Set the model to evaluation mode


## 6. Train
Implement the training loop, including loss functions and optimizers.

In [ ]:
# Training loop and loss computation

## 7. Test
Test the trained model and evaluate performance metrics.

In [3]:
# Evaluation metrics and performance analysis